In [ ]:
%matplotlib inline
%config InlineBackend.figure_format ='retina'

import torch
import socialforce

_ = torch.manual_seed(42)

# Diamond

## Parametric

This is an extension of the {ref}`pedped-2d` example to study the robustness
of the inference process to potentials with gradients that change orientation.
We use a modified $V(b)$ potential that could be described as a "diamond"
of height $V_0$ and with a half-width of $\sigma$:
\begin{align}
    V(b) &= V_0 \max\left(0, 1 - \frac{|x_1| + |x_2|}{2\sigma} \right)
\end{align}
with its two parameters $V_0$ and $\sigma$.

In [ ]:
socialforce.FieldOfView.out_of_view_factor = 0.0

V = socialforce.potentials.PedPedPotentialDiamond(sigma=0.5)
with socialforce.show.canvas(figsize=(12, 6), ncols=2) as (ax1, ax2):
    socialforce.show.potential_2d(V, ax1)
    socialforce.show.potential_2d_grad(V, ax2)

## Scenarios

Now we use the above parametric potential in a simulation to generate synthetic
scenarios according to this potential.
We generate {ref}`Circle and ParallelOvertake scenarios <scenarios>`.

In [ ]:
circle = socialforce.scenarios.Circle(ped_ped=V)
parallel_overtake = socialforce.scenarios.ParallelOvertake(ped_ped=V)
parallel_avoidance = socialforce.scenarios.ParallelAvoidance(ped_ped=V)
scenarios = circle.generate(50) + parallel_overtake.generate(50) + parallel_avoidance.generate(150)
true_experience = socialforce.Trainer.scenes_to_experience(scenarios)

All the scenes from a fixed observer:

In [ ]:
# HIDE CODE
with socialforce.show.track_canvas() as ax:
    socialforce.show.states(ax, scenarios[-1], zorder=10)
    for scene in scenarios[:-1]:
        socialforce.show.states(ax, scene, alpha=0.1)

Parallel Overtake and Avoidance scenes from the perspective of the primary pedestrian:

In [ ]:
# HIDE CODE
with socialforce.show.track_canvas() as ax:
    socialforce.show.experience(ax, true_experience)

From the perspective of the other pedestrian:

In [ ]:
# HIDE CELL
with socialforce.show.track_canvas() as ax:
    socialforce.show.experience(ax, true_experience, reference_ped=1)

## MLP Inference

We construct an coordinate-based MLP with Fourier 
Features {cite}`tancik2020fourier,rahimi2007random`.

In [ ]:
V = socialforce.potentials.PedPedPotentialMLP2D(hidden_units=32, n_hidden_layers=3)
with socialforce.show.canvas(figsize=(12, 6), ncols=2) as (ax1, ax2):
    socialforce.show.potential_2d(V, ax1)
    socialforce.show.potential_2d_grad(V, ax2)

In [ ]:
simulator = socialforce.Simulator(ped_ped=V)
opt = torch.optim.SGD(V.parameters(), lr=0.03, momentum=0.9, nesterov=True)
# socialforce.Trainer(simulator, opt).loop(10, true_experience)  # does not work

In [ ]:
with socialforce.show.canvas(figsize=(12, 6), ncols=2) as (ax1, ax2):
    socialforce.show.potential_2d(V, ax1)
    socialforce.show.potential_2d_grad(V, ax2)

## Fourier Features

We construct an coordinate-based MLP with Fourier 
Features {cite}`tancik2020fourier,rahimi2007random`.
This coordinate-based MLP with the Fourier Feature operator FF is:
\begin{align}
    \textrm{MLP}(b, d_{\perp}, d_{\parallel}) &= \textrm{Softplus} \;\; L_{1\times64} \;\; \big[ \textrm{Softplus} \;\; L_{64\times64} \big]^3 \;\;  \textrm{Softplus} \;\; L_{64\times192} \;\; \textrm{FF}_{192 \times 3} \;\; (b, d_{\perp}, d_{\parallel})
\end{align}


In [ ]:
V = socialforce.potentials.PedPedPotentialMLP2D(
    hidden_units=64, n_hidden_layers=2, n_fourier_features=64, tanh_range=3.0, fourier_scale=3.0)
with socialforce.show.canvas(figsize=(12, 6), ncols=2) as (ax1, ax2):
    socialforce.show.potential_2d(V, ax1)
    socialforce.show.potential_2d_grad(V, ax2)

In [ ]:
# HIDE OUTPUT
simulator = socialforce.Simulator(ped_ped=V)
opt = torch.optim.SGD(V.parameters(), lr=0.03, momentum=0.9, nesterov=True)
socialforce.Trainer(simulator, opt).loop(10, true_experience)

In [ ]:
with socialforce.show.canvas(figsize=(12, 6), ncols=2) as (ax1, ax2):
    socialforce.show.potential_2d(V, ax1)
    socialforce.show.potential_2d_grad(V, ax2)

## Asymmetric

We can redo the above steps with an asymmetric version of the diamond potential.
This refines the previous model by a short training on the scenarios
generated with the modified potential.

In [ ]:
V_asym = socialforce.potentials.PedPedPotentialDiamond(sigma=0.5, asymmetry_angle=-20.0)
with socialforce.show.canvas(figsize=(12, 6), ncols=2) as (ax1, ax2):
    socialforce.show.potential_2d(V_asym, ax1)
    socialforce.show.potential_2d_grad(V_asym, ax2)

In [ ]:
circle_asym = socialforce.scenarios.Circle(ped_ped=V_asym)
parallel_overtake_asym = socialforce.scenarios.ParallelOvertake(ped_ped=V_asym, b_center=0.25)
parallel_avoidance_asym = socialforce.scenarios.ParallelAvoidance(ped_ped=V_asym, b_center=-0.25)
scenarios_asym = circle_asym.generate(50) + parallel_overtake_asym.generate(50) + parallel_avoidance_asym.generate(150)
true_experience_asym = socialforce.Trainer.scenes_to_experience(scenarios_asym)

From the perspective of the primary pedestrian:

In [ ]:
# HIDE CODE
with socialforce.show.track_canvas() as ax:
    socialforce.show.experience(ax, true_experience_asym)

In [ ]:
# HIDE OUTPUT
socialforce.Trainer(simulator, opt).loop(10, true_experience_asym)

In [ ]:
with socialforce.show.canvas(figsize=(12, 6), ncols=2) as (ax1, ax2):
    socialforce.show.potential_2d(V, ax1)
    socialforce.show.potential_2d_grad(V, ax2)